Run experiments to compare quadratic underestimators with other type of estimators.


In [2]:
import pandas as pd
import numpy as np
import interval as ival
import uvarprob as uvpr
import bnb as bnb
import sub as sub
import ivalprocessor as ivproc
import pslprocessor as pslproc
import psqeprocessor as psqproc
import gridsearch as gs
from sortedcontainers import SortedKeyList
from collections import namedtuple

Define an array of test files to process

In [3]:
test_files = ["/tmp/shek.csv"]

Define some tuning parameters

In [4]:
# Maximal number of steps
max_steps = 1e6
# Tolerance
epsilon = 1e-2
# If True - global Lipschitz constant is used
global_lipschitz_interval = True
# If True - the record value is taken from the test database
known_record = False

In [5]:
def get_initial_recval(prob):
    if known_record:
        return prob.min_f
    else:
        return float('inf')

In [6]:
TestResult = namedtuple('TestResult', ['nsteps', 'record_value'])

In [7]:
def psl(prob, sym):
    psp = pslproc.PSLProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, pslproc.PSLData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [8]:
def psqe(prob, sym):
    psp = psqproc.PSQEProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, psqproc.PSQEData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [9]:
def psl_non_sym(prob):
    return psl(prob, False)

In [10]:
def psl_sym(prob):
    return psl(prob, True)

In [11]:
def psqe_non_sym(prob):
    return psqe(prob, False)

In [12]:
def psqe_sym(prob):
    return psqe(prob, True)

Tested methods

In [13]:
test_methods = {'LLI' : psl_non_sym, 'LLC' : psl_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [14]:
# test_methods = {'LLI' : psl_non_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [15]:
def run_method(method, prob):
    result = method(prob)
    return result

In [16]:
def run_test(test):
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    print(prob.min_f)
    runs = {}
    for name, method in test_methods.items():
        result = run_method(method, prob)
#         print(name, "result :", result)
        runs[name] = result
    return runs



In [22]:
def run_tests(df):
    data = {}
    for row in df.itertuples():
        for name, tres in run_test(row).items():
            steps_key = name + '_steps'
            if data.get(steps_key) == None:
                data[steps_key] = [tres.nsteps]
            else:
                data[steps_key].append(tres.nsteps)
            record_key = name + '_record'
            if data.get(record_key) == None:
                data[record_key] = [tres.record_value]
            else:
                data[record_key].append(tres.record_value)
#     print(data)
    perf_df = pd.DataFrame(data, index = df.index)
#     print(perf_df)
#     print(perf_df.diff(axis = 1, periods = 2))
    return perf_df

In [23]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col = 'name', comment = '#')
    return data

Main testing scenario

In [28]:
for file in test_files:
    df = run_tests(read_problems(file))
    filtered_df = df[['LLC_steps', 'LLI_steps', 'QLC_steps', 'QLI_steps']]
    print(filtered_df)
    mean_data = filtered_df.mean()
    max_data = filtered_df.max()
    min_data = filtered_df.min()
#     print(mean_data['LLI_steps'])
    print("=============")
#     steps_data = pd.Series(mean_data, index = ['LLC_steps', 'LLI_steps', 'QLC_steps', 'QLI_steps'])
    print("Mean:")
    print(mean_data)
    print("Min:")
    print(min_data)
    print("Max:")
    print(max_data)
    

0.53993
1.4095
0.99992
0.41883
0.72143
0.4338
0.58908
0.51048
0.6859
0.42419
           LLC_steps  LLI_steps  QLC_steps  QLI_steps
name                                                 
rshekel_0     754797     551839      23631      18426
rshekel_1     295817     309368      16093      10822
rshekel_2     448523     402303      17752      14651
rshekel_3     818613     675551      25092      14699
rshekel_4     590765     439164      21260      16703
rshekel_5     579841     572815      30956      12087
rshekel_6     809869     438548      34323      20000
rshekel_7    1000001     914419      38638      20194
rshekel_8     819837     613046      28852      19887
rshekel_9     708655     705365      28101      18728
Mean:
LLC_steps    682671.8
LLI_steps    562241.8
QLC_steps     26469.8
QLI_steps     16619.7
dtype: float64
Min:
LLC_steps    295817
LLI_steps    309368
QLC_steps     16093
QLI_steps     10822
dtype: int64
Max:
LLC_steps    1000001
LLI_steps     914419
QLC_steps      38638
